In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def str_to_set(s):
    return set(item['name'] for item in eval(s))
    

meta = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')

#meta['original_title'] === Series
#meta ==> Dataframe

meta = meta[['id','original_title','original_language','genres']]
meta = meta.rename(columns={'id':'movieId', 'original_title' : 'title', 'original_language' : 'language'})
meta = meta.loc[meta['language']=='en',:]
meta['movieId'] = pd.to_numeric(meta['movieId'])
meta['genres'] = meta['genres'].apply(str_to_set)
meta

In [3]:
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')


keywords.loc[0,'keywords']
    
# str_to_set = lambda s: set(item['name'] for item in eval(s))
    
#     genre_set = set()
#     for item in eval(s):
#         genre_set.add(item['name'])

#     return genre_set
    
str_to_set(keywords.loc[0, 'keywords'])


keywords['keywords'] = keywords['keywords'].apply(str_to_set)

keywords = keywords.rename(columns={'id' : 'movieId'})
keywords['movieId'] = pd.to_numeric(keywords['movieId'])
keywords

In [4]:
merged = pd.merge(meta, keywords, on='movieId', how='inner')
merged

In [5]:
dk = merged.loc[merged['title'] == 'The Dark Knight'].iloc[0]
dkr = merged.loc[merged['title'] == 'The Dark Knight Rises'].iloc[0]
pd.concat([dk,dkr],axis=1).T

toy = merged.loc[merged['title'] == 'Toy Story', :].iloc[0,:]

dk_set = dk.keywords|dk.genres
dkr_set = dkr.keywords|dkr.genres
toy_set = toy.keywords|toy.genres



def jaccard_similarity(set1, set2):
    if (len(set1|set2) == 0): 
        return 0
    return len(set1&set2)/len(set1|set2)



a = jaccard_similarity(dk_set, dkr_set)
b = jaccard_similarity(dk_set, toy_set)
print(a,b)

In [6]:
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings = pd.merge(ratings[['userId', 'movieId', 'rating']], meta[['movieId','title']], on='movieId', how='inner')
ratings['movieId'] = pd.to_numeric(ratings['movieId'])
ratings



In [7]:
matrix= ratings.pivot_table(index = 'userId', columns = 'title', values = 'rating')


def pearson_similarity(a, b):
    a = a-a.mean()
    b = b-b.mean()
    
    denom = ((a**2).sum() ** 0.5) * ((b**2).sum()**0.5)
    if denom == 0 : return 0
    else : return(a*b).sum()/denom
    

    
pn = matrix['Prom Night']
dk = matrix['The Dark Knight']

pearson_similarity(pn,dk)

In [8]:
#하려는 것 : The Dark Knight와 모든 영화 사이의 Pearson, Jaccard 유사도 구하기


def find_similar_movies(input_title, matrix, alpha = 0.5, k = 10):

    input_meta = merged.loc[merged['title'] == input_title, :].iloc[0,:]
    input_set = input_meta['genres'] | input_meta['keywords']

    results = []

    input_vector = matrix[input_title]


    for this_title in matrix.columns:
        if this_title == input_title:
            continue
        
        this_vector = matrix[this_title]

        this_meta = merged.loc[merged['title'] == this_title, :].iloc[0,:]
        this_set = this_meta['genres'] | this_meta['keywords']

        jaccard= jaccard_similarity(input_set, this_set)
        pearson = pearson_similarity(input_vector, this_vector)

        score = alpha * pearson + (1-alpha) * jaccard

        results.append((this_title, pearson, jaccard, score))

    results.sort(key = lambda item : item[3], reverse=True)
    return pd.DataFrame(results[:k], columns=['title','jaccard','pearson','score'])

In [9]:
find_similar_movies('Rocky III', matrix, 0.3, 20)
result = find_similar_movies('The Dark Knight', matrix, 0.3, 10)
result